In [2]:
from typing import List, Dict, Optional, Union, TypedDict
from datetime import datetime
import asyncio
from enum import Enum
from pydantic import BaseModel, Field
from sqlalchemy import create_engine, Column, String, JSON, DateTime, Integer, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import json
import numpy as np
from collections import deque

Base = declarative_base()

class CommandExecutionRecord(Base):
    __tablename__ = 'command_executions'
    
    id = Column(Integer, primary_key=True)
    command = Column(String)
    timestamp = Column(DateTime, default=datetime.utcnow)
    stdout = Column(String)
    stderr = Column(String)
    syslog = Column(String)
    system_state_before = Column(JSON)
    system_state_after = Column(JSON)
    interaction_history = Column(JSON)
    agent_decisions = Column(JSON)
    outcome_metrics = Column(JSON)
    learning_annotations = Column(JSON)

class LearningExperience(Base):
    __tablename__ = 'learning_experiences'
    
    id = Column(Integer, primary_key=True)
    agent_id = Column(String)
    command_execution_id = Column(Integer, ForeignKey('command_executions.id'))
    expertise_area = Column(String)
    success_metrics = Column(Float)
    learned_patterns = Column(JSON)
    improvement_areas = Column(JSON)
    timestamp = Column(DateTime, default=datetime.utcnow)

class SystemStateSnapshot(TypedDict):
    cpu_usage: float
    memory_usage: float
    disk_usage: Dict[str, float]
    network_status: Dict[str, str]
    running_processes: List[str]
    error_logs: List[str]

class LearningMetrics(BaseModel):
    command_success_rate: float = 0.0
    error_recovery_rate: float = 0.0
    interaction_efficiency: float = 0.0
    pattern_recognition_score: float = 0.0
    knowledge_application_rate: float = 0.0

class LearningMemory:
    def __init__(self, max_size: int = 1000):
        self.experiences = deque(maxlen=max_size)
        self.pattern_frequency = {}
        self.success_patterns = {}
        self.error_patterns = {}
        
    def add_experience(self, experience: Dict):
        self.experiences.append(experience)
        self._update_patterns(experience)
    
    def _update_patterns(self, experience: Dict):
        """Update pattern recognition based on new experience"""
        if experience.get('success', False):
            pattern = self._extract_pattern(experience)
            self.success_patterns[pattern] = self.success_patterns.get(pattern, 0) + 1
        else:
            error_pattern = self._extract_error_pattern(experience)
            self.error_patterns[error_pattern] = self.error_patterns.get(error_pattern, 0) + 1

    def _extract_pattern(self, experience: Dict) -> str:
        """Extract success pattern from experience"""
        # Implement pattern extraction logic
        return json.dumps(sorted(experience.get('key_factors', [])))

    def _extract_error_pattern(self, experience: Dict) -> str:
        """Extract error pattern from experience"""
        # Implement error pattern extraction logic
        return json.dumps(sorted(experience.get('error_factors', [])))

class LearningProgrammerAgent(BaseAgent):
    def __init__(self, 
                 name: str, 
                 expertise: List[str],
                 model_path: str,
                 db_session: sessionmaker):
        super().__init__(name, model_path)
        self.expertise = expertise
        self.db_session = db_session
        self.learning_memory = LearningMemory()
        self.learning_metrics = LearningMetrics()
        self.current_learning_focus = None
        self.learning_queue = asyncio.Queue()

    async def learn_from_execution(self, execution_record: CommandExecutionRecord) -> None:
        """Learn from command execution experience"""
        # Extract learning patterns
        patterns = self._extract_learning_patterns(execution_record)
        
        # Create learning experience record
        experience = LearningExperience(
            agent_id=self.name,
            command_execution_id=execution_record.id,
            expertise_area=self._determine_expertise_area(execution_record),
            success_metrics=self._calculate_success_metrics(execution_record),
            learned_patterns=patterns,
            improvement_areas=self._identify_improvement_areas(execution_record)
        )
        
        # Store in database
        with self.db_session() as session:
            session.add(experience)
            session.commit()
        
        # Update learning memory
        self.learning_memory.add_experience({
            'command': execution_record.command,
            'patterns': patterns,
            'success': experience.success_metrics > 0.7,
            'key_factors': self._extract_key_factors(execution_record)
        })
        
        # Update metrics
        await self._update_learning_metrics(experience)

    def _extract_learning_patterns(self, record: CommandExecutionRecord) -> Dict:
        """Extract patterns from command execution"""
        return {
            'command_patterns': self._analyze_command_patterns(record.command),
            'output_patterns': self._analyze_output_patterns(record.stdout, record.stderr),
            'state_transition_patterns': self._analyze_state_transitions(
                record.system_state_before, 
                record.system_state_after
            ),
            'interaction_patterns': self._analyze_interactions(record.interaction_history)
        }

    def _analyze_command_patterns(self, command: str) -> Dict:
        """Analyze command structure and parameters"""
        return {
            'structure': self._extract_command_structure(command),
            'parameters': self._extract_parameters(command),
            'common_patterns': self._identify_common_patterns(command)
        }

    def _analyze_output_patterns(self, stdout: str, stderr: str) -> Dict:
        """Analyze command output patterns"""
        return {
            'success_indicators': self._extract_success_indicators(stdout),
            'error_patterns': self._extract_error_patterns(stderr),
            'progress_indicators': self._extract_progress_indicators(stdout)
        }

    def _analyze_state_transitions(self, 
                                 before: SystemStateSnapshot, 
                                 after: SystemStateSnapshot) -> Dict:
        """Analyze system state changes"""
        return {
            'resource_changes': self._analyze_resource_changes(before, after),
            'status_transitions': self._analyze_status_transitions(before, after),
            'critical_changes': self._identify_critical_changes(before, after)
        }

    async def _update_learning_metrics(self, experience: LearningExperience) -> None:
        """Update agent's learning metrics"""
        # Update success rate
        self.learning_metrics.command_success_rate = (
            0.9 * self.learning_metrics.command_success_rate +
            0.1 * experience.success_metrics
        )
        
        # Update pattern recognition score
        pattern_score = len(experience.learned_patterns) / 10  # normalized score
        self.learning_metrics.pattern_recognition_score = (
            0.9 * self.learning_metrics.pattern_recognition_score +
            0.1 * pattern_score
        )
        
        # Store metrics update
        with self.db_session() as session:
            session.add(CommandExecutionRecord(
                command="metrics_update",
                agent_decisions={'metrics_update': self.learning_metrics.dict()},
                learning_annotations={
                    'pattern_score': pattern_score,
                    'success_metrics': experience.success_metrics
                }
            ))
            session.commit()

    async def adapt_to_patterns(self) -> None:
        """Adapt behavior based on learned patterns"""
        while True:
            # Get most frequent success patterns
            top_patterns = sorted(
                self.learning_memory.success_patterns.items(),
                key=lambda x: x[1],
                reverse=True
            )[:5]
            
            # Adjust behavior based on successful patterns
            for pattern, frequency in top_patterns:
                await self._incorporate_pattern(pattern)
            
            # Sleep before next adaptation cycle
            await asyncio.sleep(60)

    async def _incorporate_pattern(self, pattern: str) -> None:
        """Incorporate successful pattern into agent's behavior"""
        pattern_data = json.loads(pattern)
        
        # Update agent's decision making based on pattern
        self.knowledge_base.update({
            'successful_patterns': pattern_data
        })
        
        # Record pattern incorporation
        with self.db_session() as session:
            session.add(LearningExperience(
                agent_id=self.name,
                expertise_area='pattern_adaptation',
                learned_patterns={'incorporated_pattern': pattern_data},
                success_metrics=1.0
            ))
            session.commit()

class LearningOrchestrator:
    def __init__(self, db_session: sessionmaker):
        self.db_session = db_session
        self.agents: Dict[str, LearningProgrammerAgent] = {}
        self.learning_tasks = []

    def add_agent(self, agent: LearningProgrammerAgent) -> None:
        """Add a learning agent to the orchestrator"""
        self.agents[agent.name] = agent
        self.learning_tasks.append(
            asyncio.create_task(self._manage_agent_learning(agent))
        )

    async def _manage_agent_learning(self, agent: LearningProgrammerAgent) -> None:
        """Manage learning process for an agent"""
        while True:
            # Get recent executions
            with self.db_session() as session:
                recent_executions = session.query(CommandExecutionRecord)\
                    .order_by(CommandExecutionRecord.timestamp.desc())\
                    .limit(10)\
                    .all()

            # Process each execution for learning
            for execution in recent_executions:
                await agent.learn_from_execution(execution)

            # Adapt to learned patterns
            await agent.adapt_to_patterns()
            
            # Sleep before next learning cycle
            await asyncio.sleep(300)  # 5 minutes between learning cycles

    async def analyze_learning_progress(self) -> Dict:
        """Analyze learning progress across all agents"""
        progress_metrics = {}
        
        for agent_name, agent in self.agents.items():
            progress_metrics[agent_name] = {
                'metrics': agent.learning_metrics.dict(),
                'top_patterns': self._get_top_patterns(agent),
                'improvement_areas': self._analyze_improvement_areas(agent)
            }
        
        return progress_metrics

    def _get_top_patterns(self, agent: LearningProgrammerAgent) -> Dict:
        """Get top successful and error patterns for an agent"""
        return {
            'success_patterns': dict(sorted(
                agent.learning_memory.success_patterns.items(),
                key=lambda x: x[1],
                reverse=True
            )[:5]),
            'error_patterns': dict(sorted(
                agent.learning_memory.error_patterns.items(),
                key=lambda x: x[1],
                reverse=True
            )[:5])
        }

# Example usage
async def main():
    # Initialize database
    engine = create_engine('sqlite:///agent_learning.db')
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    
    # Create learning orchestrator
    orchestrator = LearningOrchestrator(Session)
    
    # Create learning agents
    frontend_agent = LearningProgrammerAgent(
        name="frontend_dev",
        expertise=["frontend", "ui"],
        model_path="small_model_path",
        db_session=Session
    )
    
    backend_agent = LearningProgrammerAgent(
        name="backend_dev",
        expertise=["backend", "database"],
        model_path="small_model_path",
        db_session=Session
    )
    
    # Add agents to orchestrator
    orchestrator.add_agent(frontend_agent)
    orchestrator.add_agent(backend_agent)
    
    # Run learning system
    while True:
        progress = await orchestrator.analyze_learning_progress()
        print("Learning Progress:", json.dumps(progress, indent=2))
        await asyncio.sleep(3600)  # Analysis every hour

if __name__ == "__main__":
    await main()

<ipython-input-2-12c81ffecac6>:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


NameError: name 'MessageType' is not defined